In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import datetime as dt
from matplotlib import pyplot as plt
pd.set_option('display.max_columns', None)

In [2]:
# Test data sets
benef = pd.read_csv('./data/Test_Beneficiary.csv')
inpat = pd.read_csv('./data/Test_Inpatient.csv')
outpat = pd.read_csv('./data/Test_Outpatient.csv')
fraud = pd.read_csv('./data/Test-Potential Fraud.csv')

In [3]:
print(benef.shape)
print(inpat.shape)
print(outpat.shape)
print(fraud.shape)

(63968, 25)
(9551, 30)
(125841, 27)
(1353, 1)


In [4]:
#Create columns for inpatient and outpatient data.
inpat['patientType'] = np.repeat('inpatient', len(inpat))
outpat['patientType'] = np.repeat('outpatient', len(outpat))

In [5]:
#Combining the inpatient and outpatient data
patient = pd.concat([inpat, outpat], axis=0)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [6]:
# Combining patient dataset with beneficiary dataset 
full_data =pd.merge(patient, benef, on='BeneID', how='left')

In [7]:
full_data1 = full_data.copy()

In [8]:
# Creating DaysAdmitted Feature by substracting claim start date from claim end date 
full_data1['ClaimStartDt'] = pd.to_datetime(full_data1['ClaimStartDt'])
full_data1['ClaimEndDt'] = pd.to_datetime(full_data1['ClaimEndDt'])
full_data1['DaysAdmitted'] = full_data1['ClaimEndDt'] - full_data1['ClaimStartDt']

In [9]:
# Chaging DaysAdmitted into integer object 
full_data1['DaysAdmitted'] = full_data1['DaysAdmitted'].astype(str)
full_data1['DaysAdmitted'] = full_data1['DaysAdmitted'].str.replace('days', ' ')

full_data1['DaysAdmitted'] = full_data1['DaysAdmitted']. \
str.replace('00:00:00.000000000', '')

full_data1['DaysAdmitted'] = full_data1['DaysAdmitted'].astype(int)

full_data1['DaysAdmitted'] = full_data1['DaysAdmitted'] + 1

#### First Feature Engineered: DaysAdmittedMean

In [10]:
# Creating a separate dataframe of DaysAdmittedMean by Provider
days = pd.DataFrame(full_data1.groupby('Provider')['DaysAdmitted'].mean())
days = days.reset_index()

In [11]:
# Merging Potential Fraud dataset with DaysAdmittedMean dataset 
fraud1 = fraud.copy()
fraud1 = pd.merge(fraud1, days, on='Provider')

#### 2. DeductibleAmountPaidMean

In [12]:
DeductibleMean = pd.DataFrame(full_data1.groupby('Provider') \
                              ['DeductibleAmtPaid'].mean())

fraud1 = pd.merge(fraud1, DeductibleMean, on='Provider')

#### 3. Insurance Claim Amount Reimbursed Mean

In [13]:
InscReimbursed = pd.DataFrame(full_data1.groupby('Provider') \
                              ['InscClaimAmtReimbursed'].mean())

fraud1 = pd.merge(fraud1, InscReimbursed, on='Provider')

#### 4. Insurance covered percentage mean

In [14]:
full_data1['InscCoveredPercent'] = full_data1['InscClaimAmtReimbursed'] \
/(full_data1['InscClaimAmtReimbursed'] + full_data1['DeductibleAmtPaid'])

InscCoveredPercent = pd.DataFrame(full_data1.groupby('Provider') \
                                  ['InscCoveredPercent'].mean())

fraud1 = pd.merge(fraud1, InscCoveredPercent, on='Provider')

#### 5. Total Claim Amount Mean

In [15]:
full_data1['TotalClaimAmount'] = full_data1['InscClaimAmtReimbursed']\
+ full_data1['DeductibleAmtPaid']

TotalClaimAmount = pd.DataFrame(full_data1.groupby('Provider') \
                                ['TotalClaimAmount'].mean())

fraud1 = pd.merge(fraud1, TotalClaimAmount, on='Provider')

#### 6. Daily Total Charge

In [16]:
full_data1['DailyTotalCharge'] = full_data1['TotalClaimAmount'] \
/full_data1['DaysAdmitted']

DailyTotalCharge = pd.DataFrame(full_data1.groupby('Provider') \
                                ['DailyTotalCharge'].mean())

fraud1 = pd.merge(fraud1, DailyTotalCharge, on='Provider')

#### 7-10

In [17]:
IPAnnReimbMean = pd.DataFrame(full_data1.groupby('Provider') \
                              ['IPAnnualReimbursementAmt'].mean())

IPAnnDeductMean = pd.DataFrame(full_data1.groupby('Provider') \
                               ['IPAnnualDeductibleAmt'].mean())

OPAnnReimbMean = pd.DataFrame(full_data1.groupby('Provider') \
                              ['OPAnnualReimbursementAmt'].mean())

OPAnnDeductMean = pd.DataFrame(full_data1.groupby('Provider') \
                               ['OPAnnualDeductibleAmt'].mean())

In [18]:
fraud1 = pd.merge(fraud1, IPAnnReimbMean, on='Provider')
fraud1 = pd.merge(fraud1, IPAnnDeductMean, on='Provider')
fraud1 = pd.merge(fraud1, OPAnnReimbMean, on='Provider')
fraud1 = pd.merge(fraud1, OPAnnDeductMean, on='Provider')

#### 11. Total Annual Claim Amount Mean

In [19]:
full_data1['TotalAnnClaimAmt'] = full_data1['IPAnnualReimbursementAmt']+ \
full_data1['IPAnnualDeductibleAmt']+full_data1['OPAnnualReimbursementAmt'] \
+ full_data1['OPAnnualDeductibleAmt']

TotalAnnClaimAmt = pd.DataFrame(full_data1.groupby('Provider') \
                                ['TotalAnnClaimAmt'].mean())

fraud1 = pd.merge(fraud1, TotalAnnClaimAmt, on='Provider')

#### 12. Number of Doctors per provider 

In [20]:
doctors= full_data1.groupby(['Provider', 'AttendingPhysician']) \
['AttendingPhysician'].count().reset_index(name='NumOfDoctors'). \
groupby('Provider')['NumOfDoctors'].count().reset_index()

fraud1 = pd.merge(fraud1, doctors, on='Provider')

#### 13. Number of patients per provider

In [21]:
patient= full_data.groupby(['Provider','BeneID'])['BeneID'].count(). \
reset_index(name='NumOfPatients').groupby('Provider')['NumOfPatients']. \
count().reset_index()

fraud1 = pd.merge(fraud1, patient, on='Provider')

#### 14. Type of Service Done by each provider - Inpatient/Outpatient/Both

In [22]:
a = full_data1.groupby(['Provider', 'patientType'])['BeneID'].count().reset_index(name='count').\
drop('count', axis=1)

a_list= list(zip(a.Provider, a.patientType))

providerDict= {}

for ele in a_list:
    if ele[0] not in providerDict:
        providerDict[ele[0]]= ele[1]    
    else:
        providerDict[ele[0]] = 'both'
        

providerService = pd.DataFrame(providerDict.keys(), providerDict.values()).reset_index().\
rename(columns={'index':'ServiceType', 0:'Provider'})

In [23]:
fraud1 = pd.merge(fraud1, providerService, on='Provider')

#### 15. Number of claims per provider

In [24]:
claims= full_data1.groupby(['Provider', 'ClaimID'])['ClaimID'].count().\
reset_index(name='NumOfClms').groupby('Provider')['NumOfClms'].count().reset_index()

fraud1 = pd.merge(fraud1, claims, on='Provider')

#### 16. Weekly Claims

In [25]:
full_data1['startClaimWeek']= full_data1['ClaimStartDt'].dt.week

weeklyClaims= full_data1.groupby(['Provider','startClaimWeek', 'ClaimID'])['ClaimID'].count().reset_index(name='WeeklyClaims').\
groupby(['Provider'])['WeeklyClaims'].count().reset_index()

fraud1= pd.merge(fraud1, weeklyClaims, on='Provider', how='left')

#### 17. Monthtly Claims

In [26]:
full_data1['startClaimMonth'] = full_data1['ClaimStartDt'].dt.month

monthly = full_data1.groupby(['Provider','startClaimMonth', 'ClaimID'])['ClaimID'].count().reset_index(name='MonthlyClaims').\
groupby(['Provider'])['MonthlyClaims'].count().reset_index()

fraud1 = pd.merge(fraud1, monthly, on='Provider', how='left')

#### 18. Number of states each provider operates in

In [27]:
numStates = full_data1.groupby(['Provider', 'State'])['State'].count().reset_index(name='a').groupby('Provider')['State'].count().\
reset_index(name='numStates')

fraud1 = pd.merge(fraud1, numStates, on='Provider')

#### 19. Number of countries each provider operates in 

In [28]:
numCounties = full_data.groupby(['Provider', 'County'])['County'].count().reset_index(name='a'). \
groupby('Provider')['County'].count().reset_index(name='numCounties')

fraud1 = pd.merge(fraud1, numCounties, on='Provider')

#### 20. Average number of chronic conditions per patient per provider 

In [29]:
patientChronic = full_data1.filter(regex='Chronic').replace(to_replace=2, value=0).sum(axis=1).\
reset_index(name='NumChronicCond')

chronic = pd.concat([patientChronic, full_data1[['Provider', 'ClaimID']]], axis=1)

avgChronic = chronic.groupby(['Provider', 'NumChronicCond'])['ClaimID'].count().reset_index(name='NumClaims').\
groupby('Provider').agg({'NumChronicCond':'mean', 'NumClaims':'mean'}).reset_index().\
rename(columns={'NumChronicCond':'AvgChronic', 'NumClaims':'AvgClaim'})

fraud1 = pd.merge(fraud1, avgChronic, on='Provider')

#### 21. Number of claim admit diagnosis codes per provider 

In [30]:
numDiffDiagnosisCode= full_data1.groupby(['Provider', 'ClmAdmitDiagnosisCode'])['ClmAdmitDiagnosisCode'].count(). \
reset_index(name='a').groupby('Provider')['ClmAdmitDiagnosisCode'].count().reset_index(name='numDiffDiagnosisCode')

fraud1 = pd.merge(fraud1, numDiffDiagnosisCode, on='Provider', how='left')

#### 22. Number of group diagnosis codes per provider

In [31]:
numDiffGroupDiagCode = full_data1.groupby(['Provider', 'DiagnosisGroupCode'])['DiagnosisGroupCode']. \
count().reset_index(name='a').groupby('Provider')['DiagnosisGroupCode']. \
count().reset_index(name='numDiffGroupDiagCode')

fraud1 = pd.merge(fraud1, numDiffGroupDiagCode, on='Provider', how='left')

#### 23. Average patients' age per provider

In [32]:
# Calculating the age of the patient at the time of their service
full_data1['ClaimStartDt2'] =  pd.to_datetime(full_data1['ClaimStartDt'], format='%Y/%m/%d')
full_data1['DOB'] =  pd.to_datetime(full_data1['DOB'], format='%Y/%m/%d')
full_data1['AgeWhenServed'] = full_data1['ClaimStartDt'] - full_data1['DOB']

In [33]:
# Converting the unit of AgeWhenServed to year from days
full_data1['AgeWhenServed'] = full_data1['AgeWhenServed'] / np.timedelta64(1, 'Y')

In [34]:
# Converting AgeWhenServed to int from float
full_data1['AgeWhenServed'] = full_data1['AgeWhenServed'].fillna(-1)
full_data1['AgeWhenServed'] = full_data1['AgeWhenServed'].astype(int)
full_data1['AgeWhenServed'] = full_data1['AgeWhenServed'].replace(-1, np.nan)

In [35]:
avgage = full_data1.groupby(['Provider', 'BeneID'])['AgeWhenServed']. \
mean().reset_index(name = "AvgAgeWhenServed").dropna() \
.groupby('Provider')['AvgAgeWhenServed'].mean().reset_index()

In [36]:
fraud1 = pd.merge(fraud1, avgage, on='Provider', how='left')

#### 24. Count of duplicated BeneID per provider

In [37]:
inpatient = full_data1.loc[full_data1.patientType == 'inpatient']
outpatient = full_data1.loc[full_data1.patientType == 'outpatient']

In [38]:
# Dataframe of duplicated data for inpatients
inpatient_d = inpatient[inpatient.duplicated(['ClmAdmitDiagnosisCode', \
                                              'DiagnosisGroupCode', \
                                              'ClmDiagnosisCode_1', \
                                              'ClmDiagnosisCode_2', \
                                              'ClmDiagnosisCode_3', \
                                              'ClmDiagnosisCode_4', \
                                              'ClmDiagnosisCode_5', \
                                              'ClmDiagnosisCode_6', \
                                              'ClmDiagnosisCode_7', \
                                              'ClmDiagnosisCode_8', \
                                              'ClmDiagnosisCode_9', \
                                              'ClmDiagnosisCode_10', \
                                              'ClmProcedureCode_1', \
                                              'ClmProcedureCode_2', \
                                              'ClmProcedureCode_3', \
                                              'ClmProcedureCode_4', \
                                              'ClmProcedureCode_5', \
                                              'ClmProcedureCode_6'], \
                                             keep = False)]

In [39]:
# Dataframe of duplicated data for inpatients
outpatient_d = outpatient[outpatient.duplicated(['ClmAdmitDiagnosisCode', \
                                              'DiagnosisGroupCode', \
                                              'ClmDiagnosisCode_1', \
                                              'ClmDiagnosisCode_2', \
                                              'ClmDiagnosisCode_3', \
                                              'ClmDiagnosisCode_4', \
                                              'ClmDiagnosisCode_5', \
                                              'ClmDiagnosisCode_6', \
                                              'ClmDiagnosisCode_7', \
                                              'ClmDiagnosisCode_8', \
                                              'ClmDiagnosisCode_9', \
                                              'ClmDiagnosisCode_10', \
                                              'ClmProcedureCode_1', \
                                              'ClmProcedureCode_2', \
                                              'ClmProcedureCode_3', \
                                              'ClmProcedureCode_4', \
                                              'ClmProcedureCode_5', \
                                              'ClmProcedureCode_6'], \
                                             keep = False)]

In [40]:
# totalpatient_d = Duplicated inpatient + Duplicated outpatient
totalpatient_d = pd.concat([inpatient_d, outpatient_d], axis=0)

In [41]:
duplicatedBeneID = totalpatient_d.groupby('Provider')['BeneID'].value_counts().reset_index(name = "ValueCount")
duplicatedBeneID = duplicatedBeneID.loc[duplicatedBeneID['ValueCount'] > 1]
duplicatedBeneID = duplicatedBeneID.groupby('Provider')['BeneID'].count().reset_index(name = "NumOfDuplicatedBeneID")

In [42]:
fraud1 = pd.merge(fraud1, duplicatedBeneID, on='Provider', how='left')

#### 25. Count of duplicated attending physicians per provider

In [43]:
duplicatedPhysician = totalpatient_d.groupby('Provider')['AttendingPhysician'].value_counts().reset_index(name = "ValueCount")
duplicatedPhysician = duplicatedPhysician.loc[duplicatedPhysician['ValueCount'] > 1]
duplicatedPhysician = duplicatedPhysician.groupby('Provider')['AttendingPhysician'].count().reset_index(name = "NumOfDuplicatedAttendingPhysician")

In [44]:
fraud1 = pd.merge(fraud1, duplicatedPhysician, on='Provider', how='left')

#### 26. Count of duplicated claims per provider

In [45]:
duplicatedClaims = totalpatient_d.groupby('Provider').agg({'ClaimID' : 'count'}).reset_index()
duplicatedClaims = duplicatedClaims.rename({'ClaimID' : 'NumOfDuplicatedClaims'}, axis=1)

In [46]:
fraud1 = pd.merge(fraud1, duplicatedClaims, on='Provider', how='left')

#### 27 - 40

In [47]:
full_data2 = full_data1.copy()

In [48]:
full_data2 = full_data2.replace({'RenalDiseaseIndicator' : 'Y'}, 1)

full_data2['RenalDiseaseIndicator'] = full_data2 \
['RenalDiseaseIndicator'].astype(int)

In [49]:
full_data2 = full_data2.replace({'ChronicCond_Alzheimer' : 2, \
                                 'ChronicCond_Heartfailure' : 2, \
                                 'ChronicCond_KidneyDisease' : 2, \
                                 'ChronicCond_Cancer' : 2, \
                                 'ChronicCond_ObstrPulmonary' : 2, \
                                 'ChronicCond_Depression' : 2, \
                                 'ChronicCond_Diabetes' : 2, \
                                 'ChronicCond_IschemicHeart' : 2, \
                                 'ChronicCond_Osteoporasis' : 2, \
                                 'ChronicCond_rheumatoidarthritis' : 2, \
                                 'ChronicCond_stroke' : 2}, 0)

In [50]:
full_data2['NumChronicCond'] = full_data2['ChronicCond_Alzheimer'] \
+full_data2['ChronicCond_Heartfailure']+ \
full_data2['ChronicCond_KidneyDisease']+ \
full_data2['ChronicCond_Cancer']+ \
full_data2['ChronicCond_ObstrPulmonary']+ \
full_data2['ChronicCond_Depression']+ \
full_data2['ChronicCond_Diabetes']+ \
full_data2['ChronicCond_IschemicHeart']+ \
full_data2['ChronicCond_Osteoporasis']+ \
full_data2['ChronicCond_rheumatoidarthritis']+ \
full_data2['ChronicCond_stroke']+ \
full_data2['RenalDiseaseIndicator']

In [51]:
chronicConditions = full_data2.groupby(['Provider', 'BeneID'])['NumChronicCond'].value_counts().reset_index(name = 'NumChronicCondCount')
chronicConditions['NumChronicCondCount'] = chronicConditions['NumChronicCondCount'].apply(lambda x: 1 if x >= 1 else 0)
chronicConditions = chronicConditions.groupby('Provider')['NumChronicCond'].value_counts().reset_index(name = 'NumChronicCondCount')

In [52]:
chronicConditions = chronicConditions.groupby(['Provider', 'NumChronicCond']).NumChronicCondCount.sum()
chronicConditions = chronicConditions.unstack(level='NumChronicCond').reset_index()
chronicConditions = chronicConditions.rename({0 : 'ChronicCond0', 
                                              1 : 'ChronicCond1', 
                                              2 : 'ChronicCond2',
                                              3 : 'ChronicCond3',
                                              4 : 'ChronicCond4',
                                              5 : 'ChronicCond5',
                                              6 : 'ChronicCond6',
                                              7 : 'ChronicCond7',
                                              8 : 'ChronicCond8',
                                              9 : 'ChronicCond9',
                                              10 : 'ChronicCond10',
                                              11 : 'ChronicCond11',
                                              12 : 'ChronicCond12'}, axis=1)
chronicConditions.fillna(0, inplace=True)

In [53]:
fraud1 = pd.merge(fraud1, chronicConditions, \
                  on = 'Provider', how = 'left')

#### 41-42 Counts of each gender of patients per provider

In [54]:
gender = full_data1.groupby(['Provider', 'BeneID', 'Gender'])['Gender'].count().reset_index(name = 'GenderCount')
gender['GenderCount'] = gender['GenderCount'].apply(lambda x: 1 if x >= 1 else 0)
gender = gender.groupby('Provider')['Gender'].value_counts().reset_index(name = 'GenderCount')

In [55]:
gender['Gender1'] = 0
gender['Gender2'] = 0

for index in gender.index:
    if gender['Gender'][index] == 1:
        gender['Gender1'][index] += gender['GenderCount'][index]
    if gender['Gender'][index] == 2:
        gender['Gender2'][index] += gender['GenderCount'][index]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [56]:
gender = gender.drop('Gender', axis = 1)
gender = gender.drop('GenderCount', axis = 1)

In [57]:
genderCount = gender.groupby('Provider'). \
agg({'Gender1' : 'sum', 'Gender2' : 'sum'}).reset_index()

In [58]:
fraud1 = pd.merge(fraud1, genderCount, on = 'Provider', how = 'left')

#### 43-46. Counts of each race of patients per provider

In [59]:
race = full_data1.groupby(['Provider', 'BeneID', 'Race'])['Race'].count().reset_index(name = 'RaceCount')
race['RaceCount'] = race['RaceCount'].apply(lambda x: 1 if x >= 1 else 0)
race = race.groupby('Provider')['Race'].value_counts().reset_index(name = 'RaceCount')

In [60]:
race['Race1'] = 0
race['Race2'] = 0
race['Race3'] = 0
race['Race5'] = 0

for index in race.index:
    if race['Race'][index] == 1:
        race['Race1'][index] += race['RaceCount'][index]
    if race['Race'][index] == 2:
        race['Race2'][index] += race['RaceCount'][index]
    if race['Race'][index] == 3:
        race['Race3'][index] += race['RaceCount'][index]
    if race['Race'][index] == 5:
        race['Race5'][index] += race['RaceCount'][index]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/i

In [61]:
race = race.drop('Race', axis = 1)
race = race.drop('RaceCount', axis = 1)

In [62]:
raceCount = race.groupby('Provider'). \
agg({'Race1' : 'sum', 'Race2' : 'sum', 'Race3' : 'sum', \
     'Race5' : 'sum'}).reset_index()

In [63]:
fraud1 = pd.merge(fraud1, raceCount, on = 'Provider', how = 'left')

#### Impute na values with 0

In [64]:
fraud1 = fraud1.replace(np.nan,0)

#### Pickling Dataset

In [95]:
fraud1.to_pickle('test_dataset.pkl')

## Feature Engineering: Round 2
### - After Performing Random Trees Feature Selection on Train Data

In [67]:
test_data = fraud1.copy()

In [68]:
#dropping columns after performing Random Trees feature importance on train

test_data.drop(['WeeklyClaims', 'MonthlyClaims', 'IPAnnualReimbursementAmt', 'IPAnnualDeductibleAmt',
                'OPAnnualReimbursementAmt', 'OPAnnualDeductibleAmt', 'DeductibleAmtPaid',
                'InscClaimAmtReimbursed'], axis=1, inplace=True)

#### New Feature: Number of distinct operating physicians per provider

In [70]:
opPhy = full_data1.groupby(['Provider', 'OperatingPhysician'])['OperatingPhysician'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincOpPhy')

In [72]:
test_data = pd.merge(test_data, opPhy, on = 'Provider', how = 'left')

#### Number of Distinct "Other Physicians" per provider 

In [74]:
otherPhy = full_data1.groupby(['Provider', 'OtherPhysician'])['OtherPhysician'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincOtherPhy')

In [75]:
test_data = pd.merge(test_data, otherPhy, on = 'Provider', how = 'left')

#### Number of Distinct Claim Diagnosis Codes per provider

In [77]:
diagCode1 = full_data1.groupby(['Provider', 'ClmDiagnosisCode_1'])['ClmDiagnosisCode_1'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmDiagCode1')

diagCode2 = full_data1.groupby(['Provider', 'ClmDiagnosisCode_2'])['ClmDiagnosisCode_2'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmDiagCode2')

diagCode3 = full_data1.groupby(['Provider', 'ClmDiagnosisCode_3'])['ClmDiagnosisCode_3'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmDiagCode3')

diagCode4 = full_data1.groupby(['Provider', 'ClmDiagnosisCode_4'])['ClmDiagnosisCode_4'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmDiagCode4')

diagCode5 = full_data1.groupby(['Provider', 'ClmDiagnosisCode_5'])['ClmDiagnosisCode_5'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmDiagCode5')

diagCode6= full_data1.groupby(['Provider', 'ClmDiagnosisCode_6'])['ClmDiagnosisCode_6'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmDiagCode6')

diagCode7 = full_data1.groupby(['Provider', 'ClmDiagnosisCode_7'])['ClmDiagnosisCode_7'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmDiagCode7')

diagCode8 = full_data1.groupby(['Provider', 'ClmDiagnosisCode_8'])['ClmDiagnosisCode_8'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmDiagCode8')

diagCode9 = full_data1.groupby(['Provider', 'ClmDiagnosisCode_9'])['ClmDiagnosisCode_9'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmDiagCode9')

diagCode10 = full_data1.groupby(['Provider', 'ClmDiagnosisCode_10'])['ClmDiagnosisCode_10'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmDiagCode10')

In [78]:
test_data = pd.merge(test_data, diagCode1, on='Provider', how='left')
test_data = pd.merge(test_data, diagCode2, on='Provider', how='left')
test_data = pd.merge(test_data, diagCode3, on='Provider', how='left')
test_data = pd.merge(test_data, diagCode4, on='Provider', how='left')
test_data = pd.merge(test_data, diagCode5, on='Provider', how='left')
test_data = pd.merge(test_data, diagCode6, on='Provider', how='left')
test_data = pd.merge(test_data, diagCode7, on='Provider', how='left')
test_data = pd.merge(test_data, diagCode8, on='Provider', how='left')
test_data = pd.merge(test_data, diagCode9, on='Provider', how='left')
test_data = pd.merge(test_data, diagCode10, on='Provider', how='left')

#### Number of Distinct Procedure Diagnosis Codes per provider

In [79]:
proCode1= full_data1.groupby(['Provider', 'ClmProcedureCode_1'])['ClmProcedureCode_1'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmProCode1')


proCode2 = full_data1.groupby(['Provider', 'ClmProcedureCode_2'])['ClmProcedureCode_2'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmProCode2')


proCode3 = full_data1.groupby(['Provider', 'ClmProcedureCode_3'])['ClmProcedureCode_3'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmProCode3')

proCode4 = full_data1.groupby(['Provider', 'ClmProcedureCode_4'])['ClmProcedureCode_4'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmProCode4')


proCode5 = full_data1.groupby(['Provider', 'ClmProcedureCode_5'])['ClmProcedureCode_5'].count().reset_index(name='a').\
groupby('Provider')['a'].count().reset_index(name= 'NumDistincClmProCode5')

In [80]:
test_data = pd.merge(test_data, proCode1, on='Provider', how='left')
test_data = pd.merge(test_data, proCode2, on='Provider', how='left')
test_data = pd.merge(test_data, proCode3, on='Provider', how='left')
test_data = pd.merge(test_data, proCode4, on='Provider', how='left')
test_data = pd.merge(test_data, proCode5, on='Provider', how='left')

#### Impute na values with 0

In [83]:
test_data = test_data.replace(np.nan,0)

#### Pickling dataset 2

In [85]:
test_data.to_pickle('test_dataset2.pkl')

# Feature Engineering: Round 3

In [3]:
test_data3 = pd.read_pickle("test_dataset2.pkl")

In [4]:
test_data3 = test_data3.drop(columns= ['NumDistincClmDiagCode1','NumDistincClmDiagCode2','NumDistincClmDiagCode3', \
                   'NumDistincClmDiagCode4','NumDistincClmDiagCode5','NumDistincClmDiagCode6', \
                   'NumDistincClmDiagCode7','NumDistincClmDiagCode8','NumDistincClmDiagCode9', \
                   'NumDistincClmDiagCode10','NumDistincClmProCode1','NumDistincClmProCode2','NumDistincClmProCode3', \
                  'NumDistincClmProCode4','NumDistincClmProCode5'], axis=1)

In [5]:
test_data3 = test_data3.rename(columns={'numDiffDiagnosisCode':'NumUniqClmAdmitDiagCOde',
                                        'numDiffGroupDiagCode':'NumUniqGroupDiagCode'})

In [6]:
test_data3.to_pickle('test_dataset3.pkl')